In this notebook we will use techniques to enhance our dataset. first let's define required variables

In [ ]:
DATA_PATH_TO_YAML = "/content/drive/MyDrive/5 cats.v2i.yolov11 (1)/data.yaml"
INPUT_IMAGES_DIR = "/content/drive/MyDrive/5 cats.v2i.yolov11 (1)/train/images"
INPUT_LABELS_DIR = "/content/drive/MyDrive/5 cats.v2i.yolov11 (1)/train/labels"
OUTPUT_IMAGES_DIR = "/content/drive/MyDrive/5 cats.v2i.yolov11 (1)/Albumation_Aug/img"
OUTPUT_LABELS_DIR = "/content/drive/MyDrive/5 cats.v2i.yolov11 (1)/Albumation_Aug/labels"

# Package Installation

In [ ]:
!pip install albumentations
!pip install ultralytics

**Import required libraries**

In [ ]:
import albumentations as A
import ultralytics
from albumentations.pytorch import ToTensorV2
import os
import cv2
from pathlib import Path
import tqdm
import numpy as np


In [ ]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Blur(blur_limit=3, p=0.1),
    A.RandomSizedBBoxSafeCrop(height=416, width=416, p=0.5),
    A.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),
    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [ ]:
def load_yolo_labels(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()

    bboxes = []
    class_labels = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        bboxes.append([x_center, y_center, width, height])
        class_labels.append(int(class_id))

    return bboxes, class_labels


In [ ]:
def save_augmented_data(image, bboxes, labels, base_filename, aug_num):
    image_np = image.permute(1, 2, 0).numpy()
    image_np = (image_np * 255).astype(np.uint8)
    image_filename = f"{base_filename}_aug{aug_num}.jpg"
    cv2.imwrite(os.path.join(OUTPUT_IMAGES_DIR, image_filename),
                cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))

    label_filename = f"{base_filename}_aug{aug_num}.txt"
    with open(os.path.join(OUTPUT_LABELS_DIR, label_filename), 'w') as f:
        for bbox, label in zip(bboxes, labels):
            f.write(f"{label} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

In [ ]:
os.makedirs(OUTPUT_IMAGES_DIR, exist_ok=True)
os.makedirs(OUTPUT_LABELS_DIR, exist_ok=True)

In [ ]:
def load_image(pth):
  img = cv2.imread(str(pth))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

In [ ]:
def transform_and_save(img, main_bbox, main_lbls):
  my_transformed = transform(
                image=img,
                bboxes=main_bbox,
                class_labels=main_lbls
            )

  transformed_image = my_transformed["image"]
  transformed_bboxes = my_transformed["bboxes"]
  transformed_labels = my_transformed["class_labels"]
  if len(transformed_bboxes) == 0:
                return False
  save_augmented_data(
                transformed_image,
                transformed_bboxes,
                transformed_labels,
                image_path.stem,
                aug_num
            )
  return True


In [ ]:
image_paths = list(Path(INPUT_IMAGES_DIR).glob("*.*")) #get all images within the image folder

for image_path in tqdm.tqdm(image_paths, desc="Processing images"):
  image = load_image(image_path)
  label_path = Path(INPUT_LABELS_DIR) / f"{image_path.stem}.txt"
  if not label_path.exists():
            continue

  original_bboxes, original_labels = load_yolo_labels(label_path)

  for aug_num in range(2):
    transform_and_save(image, original_bboxes, original_labels)
    if not transform_and_save:
      continue
    else:
      print("successfully done epoch")

Processing images:  11%|█         | 1/9 [00:00<00:00,  9.27it/s]

successfully done epoch
successfully done epoch
successfully done epoch
successfully done epoch
successfully done epoch


Processing images:  56%|█████▌    | 5/9 [00:00<00:00, 10.72it/s]

successfully done epoch
successfully done epoch
successfully done epoch
successfully done epoch
successfully done epoch


Processing images:  78%|███████▊  | 7/9 [00:00<00:00, 11.76it/s]

successfully done epoch
successfully done epoch
successfully done epoch
successfully done epoch
successfully done epoch
successfully done epoch


Processing images: 100%|██████████| 9/9 [00:00<00:00, 11.07it/s]

successfully done epoch
successfully done epoch
